In [17]:
# import reqs
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import os
from pydub import AudioSegment
import re
import subprocess
import requests
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [19]:
# connect the video files
video_files = './video_files'
audio_files = './audio_files'
text_files = './text_files'

folders = [video_files, audio_files, text_files]
for folder in folders:
    # Check if the output folder exists
    if not os.path.exists(folder):
    # If not, create the folder
        os.makedirs(folder)

In [14]:
# extract the audio from the video and export the audio as a wav file
for video_file in os.listdir(video_files):
    if video_file.endswith('.mp4'):
        video_path = os.path.join(video_files, video_file)
        audio = AudioSegment.from_file(video_path, format="mp4")
        audio.export(os.path.join(audio_files, f"{video_file[:-4]}.wav"), format="wav")

In [16]:
# function to transcribe and save the output in txt file
def transcribe_and_save(audio_files, text_files, model='medium.en'):
    # Construct the Whisper command
    whisper_command = f"whisper '{audio_files}' --model {model}"
    
    try:
        # Run the Whisper command
        transcription = subprocess.check_output(whisper_command, shell=True, text=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        print(f"Error during transcription: {e.output}")
        return
    
    # Clean and join the sentences
    output_without_time = re.sub(r'\[\d+:\d+\.\d+ --> \d+:\d+\.\d+\]  ', '', transcription)
    sentences = [line.strip() for line in output_without_time.split('\n') if line.strip()]
    joined_text = ' '.join(sentences)

    # Create the corresponding text file name
    audio_file_name = os.path.basename(audio_files)
    text_file_name = os.path.splitext(audio_file_name)[0] + '.txt'
    file_path = os.path.join(text_files, text_file_name)

    # Save the output as a txt file
    with open(file_path, 'w') as file:
        file.write(joined_text)

    print(f'Text for {audio_file_name} has been saved to: {file_path}')


# Transcribing all the audio files in the directory
for audio_file in os.listdir(audio_files):
    if audio_file.endswith('.wav'):
        audio_files = os.path.join(audio_files, audio_file)
        transcribe_and_save(audio_files, text_files)

# This will take a while to trduge through, be patient, its a lot of data for a 5 min video

100%|█████████████████████████████████████| 1.42G/1.42G [02:37<00:00, 9.71MiB/s]
/Users/shayonkeating/anaconda3/envs/pydata/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Text for parkinsons_disease.wav has been saved to: ./text_files/parkinsons_disease.txt


### Making the knowledge graph

In [25]:
# Global Constants API endpoint, API key, prompt text
from utils import read_api_key
API_ENDPOINT = "https://api.openai.com/v1/chat/completions"
api_key_file_path = './auth/api_key.txt'
api_key = read_api_key(api_key_file_path)
prompt_text = """Based on the given prompt, identify and enumerate all possible connections, then compile a summary of updates. 
For each update involving a connection, format it as [ENTITY 1, CONNECTION, ENTITY 2]. This connection is directional, 
indicating the sequence is significant.
Example:
prompt: The Sun generates solar energy and is also responsible for Vitamin D production.
updates:
[["Sun", "generates", "solar energy"], ["Sun", "responsible for", "Vitamin D"]]
prompt: $prompt
updates:"""

In [41]:
# Graph Creation Function
def create_graph(df, rel_labels):
    G = nx.from_pandas_edgelist(df, "source", "target",
                              edge_attr=True, create_using=nx.MultiDiGraph())
    plt.figure(figsize=(12, 12))

    pos = nx.spring_layout(G)
    nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos=pos)
    nx.draw_networkx_edge_labels(
        G,
        pos,
        edge_labels=rel_labels,
        font_color='red'
    )
    plt.show()

# Data Preparation Function
def preparing_data_for_graph(api_response):
    if api_response is None:
        print("API response is None. Exiting function.")
        return None, None 
        
    response_text = api_response.text
    entity_relation_lst = json.loads(json.loads(response_text)["choices"][0]["text"])
    entity_relation_lst = [x for x in entity_relation_lst if len(x) == 3]
    source = [i[0] for i in entity_relation_lst]
    target = [i[2] for i in entity_relation_lst]
    relations = [i[1] for i in entity_relation_lst]

    kg_df = pd.DataFrame({'source': source, 'target': target, 'edge': relations})
    relation_labels = dict(zip(zip(kg_df.source, kg_df.target), kg_df.edge))
    return kg_df,relation_labels

In [27]:
# OpenAI API Call Function
def call_gpt_api(api_key, prompt_text):
    global API_ENDPOINT
    try:
        data = {
            "model": "gpt-3.5-turbo",
            "prompt": prompt_text,
            "max_tokens": 3000,
            "stop": "\n",
            "temperature": 0
        }
        headers = {"Content-Type": "application/json", "Authorization": "Bearer " + api_key}
        r = requests.post(url=API_ENDPOINT, headers=headers, json=data)
        response_data = r.json()  # Parse the response as JSON
        print("Response content:", response_data)
        return response_data
    except Exception as e:
        print("Error:", e)

In [35]:
def main(text_file_path, api_key):
    # Use text_file_path, not file_path
    with open(text_file_path, 'r') as file:
        kb_text = file.read()

    # Assuming prompt_text and the rest of your workflow is defined elsewhere correctly
    global prompt_text
    prompt_text = prompt_text.replace("$prompt", kb_text)

    api_response = call_gpt_api(api_key, prompt_text)
    df, rel_labels = preparing_data_for_graph(api_response)
    create_graph(df, rel_labels)

In [42]:
text_file_path = "./text_files/parkinsons_disease.txt"
api_key = api_key
main(text_file_path, api_key)

Error: name 'requests' is not defined
API response is None. Exiting function.


NameError: name 'nx' is not defined